In [1]:
library(xml2)
library(rvest)
library(ggplot2)
library(dplyr)



Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [2]:
# define a function : scrapping each year payrol
# given url vectors ,the function will scrape each url and it's node ,and return the data.frame 
crawler_each_year <- function(url,node,colname_node){
    rd <- read_html(url)
    node_name <- html_nodes(rd,node)
    text <- html_text(node_name)
    
    #  turn text vector into frame data.frame
    tmp_row <- character()
    frame <- data.frame(matrix(ncol = 8),stringsAsFactors=FALSE)

    for (i in 1 : length(text)){
        if(i %% 8 != 0 ){
            tmp_row <- c(tmp_row,text[i])
        }
        else{
           tmp_row <- c(tmp_row,text[i])
           frame <- rbind(frame,tmp_row)
           tmp_row <- character() 
        }
    }
    # remove the row with NA value
    frame <- frame[complete.cases(frame),]
    
    # frame is the data.frame without the colnames,colnames need to be added
    node_colname <- html_nodes(rd,colname_node)
    colname <- html_text(node_colname)
    if(length(colname) != 8){
        stop("colname is wrong !!")
    }
    colnames(frame) <- colname
    # rank the Team
    frame <- frame %>% mutate(Rank = row_number())
    return(frame)
}
# test
x <- crawler_each_year("https://www.spotrac.com/mlb/payroll/2011/",".result , td.xs-hide , .noborderleft","#payroll , #susp , #buried , #dead , #ir , #active , #players , #team")
head(x,20)                                                                                            

Team,Roster,25-Man Payroll,Injured Reserve,Retained,Buried,Suspended,2011 Total Payroll,Rank
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
New York Yankees,41,"$212,609,678",0-,"$950,000",0-,0-,"$213,559,678",1
Boston Red Sox,47,"$164,517,590",0-,"$8,239,278",0-,0-,"$172,553,261",2
Philadelphia Phillies,38,"$169,799,882",0-,"$168,300",0-,0-,"$169,968,182",3
Los Angeles Angels of Anaheim,38,"$133,613,959",0-,"$13,034,600",0-,0-,"$146,648,559",4
New York Mets,38,"$96,163,418",0-,"$45,668,423",0-,0-,"$141,831,841",5
Chicago Cubs,40,"$124,652,698",0-,"$14,825,000",0-,0-,"$139,477,698",6
Chicago White Sox,40,"$123,584,951",0-,"$7,038,798",0-,0-,"$130,623,749",7
San Francisco Giants,39,"$124,882,250",0-,"$1,100,000",0-,0-,"$125,982,250",8
Los Angeles Dodgers,41,"$98,679,615",0-,"$19,989,000",0-,0-,"$118,668,615",9


In [3]:
# I want to scrapping mulitiple page of data 
# the url problem need to be tackled
# today I just want to get the Boston Red Sox data  from 2011 to 2018
# 2015 url :https://www.spotrac.com/mlb/boston-red-sox/payroll/2015/


url_vect <- rep("https://www.spotrac.com/mlb/payroll/20",8)
tail_num <- 11:18
url_vect <- paste(url_vect,tail_num,sep = "") %>% paste("/",sep = "")
url_vect <- c(url_vect,"https://www.spotrac.com/mlb/payroll/")
print(url_vect)




[1] "https://www.spotrac.com/mlb/payroll/2011/"
[2] "https://www.spotrac.com/mlb/payroll/2012/"
[3] "https://www.spotrac.com/mlb/payroll/2013/"
[4] "https://www.spotrac.com/mlb/payroll/2014/"
[5] "https://www.spotrac.com/mlb/payroll/2015/"
[6] "https://www.spotrac.com/mlb/payroll/2016/"
[7] "https://www.spotrac.com/mlb/payroll/2017/"
[8] "https://www.spotrac.com/mlb/payroll/2018/"
[9] "https://www.spotrac.com/mlb/payroll/"     


In [4]:
each_year_payrol <- list()
tmp_frame <- data.frame()
for(i in 1:length(url_vect)){
    tmp_frame <- crawler_each_year(url_vect[i],".result , td.xs-hide , .noborderleft","#payroll , #susp , #buried , #dead , #ir , #active , #players , #team")
    
    if(i == 1){
        print(class(tmp_frame))
    }
    #each_year_payrol <- c(each_year_payrol,tmp_frame)  
    # list append data.frame is wrong,it's turn each row in dataframe into vector then append to list 
    # solution :
    each_year_payrol <- append(each_year_payrol,list(tmp_frame))  
    tmp_frame <- data.frame()
}
str(each_year_payrol)
x<- each_year_payrol


[1] "data.frame"
List of 9
 $ :'data.frame':	30 obs. of  9 variables:
  ..$ Team              : chr [1:30] "New York Yankees" "Boston Red Sox" "Philadelphia Phillies" "Los Angeles Angels of Anaheim" ...
  ..$ Roster            : chr [1:30] "41" "47" "38" "38" ...
  ..$ 25-Man Payroll    : chr [1:30] "$212,609,678" "$164,517,590" "$169,799,882" "$133,613,959" ...
  ..$ Injured Reserve   : chr [1:30] "0-" "0-" "0-" "0-" ...
  ..$ Retained          : chr [1:30] "$950,000" "$8,239,278" "$168,300" "$13,034,600" ...
  ..$ Buried            : chr [1:30] "0-" "0-" "0-" "0-" ...
  ..$ Suspended         : chr [1:30] "0-" "0-" "0-" "0-" ...
  ..$ 2011 Total Payroll: chr [1:30] "$213,559,678" "$172,553,261" "$169,968,182" "$146,648,559" ...
  ..$ Rank              : int [1:30] 1 2 3 4 5 6 7 8 9 10 ...
 $ :'data.frame':	31 obs. of  9 variables:
  ..$ Team              : chr [1:31] "New York Yankees" "Philadelphia Phillies" "Boston Red Sox" "Los Angeles Angels of Anaheim" ...
  ..$ Roster           

In [5]:
x[[1]][[1]]

[1] "New York Yankees"              "Boston Red Sox"               
 [3] "Philadelphia Phillies"         "Los Angeles Angels of Anaheim"
 [5] "New York Mets"                 "Chicago Cubs"                 
 [7] "Chicago White Sox"             "San Francisco Giants"         
 [9] "Los Angeles Dodgers"           "St. Louis Cardinals"          
[11] "Minnesota Twins"               "Detroit Tigers"               
[13] "Texas Rangers"                 "Seattle Mariners"             
[15] "Atlanta Braves"                "Baltimore Orioles"            
[17] "Milwaukee Brewers"             "Houston Astros"               
[19] "Colorado Rockies"              "Cincinnati Reds"              
[21] "Oakland Athletics"             "Washington Nationals"         
[23] "Toronto Blue Jays"             "Arizona Diamondbacks"         
[25] "Florida Marlins"               "Cleveland Indians"            
[27] "Pittsburgh Pirates"            "San Diego Padres"             
[29] "Tampa Bay Rays"                "Kansas City Royals"

In [6]:
# ouptut with excel form 
# just try 2011
getwd()
write.table(each_year_payrol[1],"2011.csv",sep=",",row.names = FALSE)

[1] "D:/jupyter notebook 工作區/棒球球員薪資數據"

In [7]:
# ouptut with excel form every year
for (i in 1 : length(each_year_payrol)){
    year <- as.character(2010 + i)
    file_name <- paste(year,".csv",sep="")
    write.table(each_year_payrol[i],file_name,sep=",",row.names = FALSE)
}